# Dataset YouTube API

In [1]:
import googleapiclient.discovery
from resources.keys import api_key as DEVELOPER_KEY
import pandas as pd
from resources.basicIO import InputOutput as IO

## API Call

In [2]:
# API CALL
api_service_name = "youtube"
api_version = "v3"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

## Fetch Channels from YouTube

In [3]:
queries = ["lakhimpur violence", "farmer protest","kisan andolan","farmer bill", 
           "agriculture reform bill 2020","kisan morcha","rakesh tikait",
           "kisan mahapanchayat","farmers march","Farmer Protest LatestNews",
           "Doubling farmers income","krishi mandi","farmlaws","Krishi Bill 2020",
           "Haryana farmers Protest","farmer protest delhi border","A Day at the Tikri Border",
           "farmers protest in Tohana","tractor rally delhi","kisan vs police","kisan riot",
           "kisan taliban","farmers bill withdrawn","kisan bill","farmer bill latest news",
           "minimum support price farm bill","farm law repealed"]
    

# lists
video_Id = []
video_Title = []
channel_Id = []
channel_name = []
video_Date = []

# take multiple search results
for i in range(len(queries)):
    request = youtube.search().list(
        part="snippet",
        maxResults=50,
        q=queries[i],
    )
    response = request.execute()
    videos_total = len(response['items'])

    # all parsed videos
    for i in range(videos_total):
        # insert if unique video id

        try:
            if response['items'][i]['id']['videoId'] not in video_Id:
                video_Id.append(response['items'][i]['id']['videoId'])
                video_Title.append(response['items'][i]['snippet']['title'])
                channel_Id.append(response['items'][i]['snippet']['channelId'])
                channel_name.append(
                    response['items'][i]['snippet']['channelTitle'])
                video_Date.append(
                    response['items'][i]['snippet']['publishedAt'])
        except:
            print('VideoId miss')

VideoId miss
VideoId miss


### Make a csv Database for search results

In [4]:
pd.DataFrame({'videoId': video_Id,
              'title': video_Title,
              'channelId': channel_Id,
              'channelName': channel_name,
              'date': video_Date}).to_csv('datasets/search_results.csv', index=False)

In [5]:
# display the dataset
output = pd.read_csv("datasets/search_results.csv")
display(output)


,videoId,title,channelId,channelName,date
0,l1oobMY7CE4,"Lakhimpur Kheri Violence - 8 people killed, Ca...",UCrC8mOqJQpoB7NuIMKIS6rQ,Study IQ education,2021-10-04T05:45:19Z
1,v_xiGjSd6aw,Watch: Full 1 minute 45 second Video of Lakhim...,UCwqusr8YDwM-3mEYTDeJHzw,Republic World,2021-10-05T09:05:09Z
2,XcMqzv3KWNA,Lakhimpur Kheri Violence: Video Of Questioning...,UCZFMm1mMw0F81Z37aaEzTUA,NDTV,2021-10-06T05:52:52Z
3,Bvcd0LK2K88,Lakhimpur Kheri Violence: 7 Shocking Videos De...,UCYPvAwZP8pZhSMW8qs7cVCw,India Today,2021-10-05T10:27:54Z
4,Gz7kaVLbvJY,DNA: लखीमपुर खीरी हिंसा में हुई मौतों का जिम्म...,UCIvaYmXn910QMdemBG3v1pQ,Zee News,2021-10-04T16:31:47Z
...,...,...,...,...,...
1071,Yf3DWsU5w9Q,First Reactions On PM Modi&#39;s Decision To R...,UCmRbHAgG2k2vDUvb3xsEunQ,CNBC-TV18,2021-11-19T06:14:00Z
1072,gtYxv5ogb74,Asaduddin Owaisi: PM swallowed his pride and r...,UCRWFSbif-RFENbBrSiez1DA,ABP NEWS,2021-11-19T09:45:10Z
1073,qqSv_ZkP2uQ,PM Modi Full Speech: &#39;New farm laws will b...,UCJFOER35ggIWwsXh2ZDnqyg,The Economic Times,2021-11-19T04:42:18Z
1074,VuxzqG1iDXI,Farm Laws Repealed| First Reaction From Former...,UCef1-8eOpJgud7szVPlZQAQ,CNN-News18,2021-11-19T04:19:34Z


## Fetch Comments from YouTube

In [3]:
search_results = pd.read_csv("datasets/search_results.csv")

# lists
user_name = []
user_comment = []
user_likes_count = []
video_id = []

for i in range(len(search_results)):
    try:
        # ith video
        videoID = (search_results['videoId'][i])

        # next pagen token
        npt = None
        pages = 20

        for j in range(pages):

            # if next page does not exist
            if(npt == None):
                request = youtube.commentThreads().list(
                part="snippet,replies",
                maxResults=100,
                order="relevance",
                videoId=videoID,
                )
            else:
                request = youtube.commentThreads().list(
                    part="snippet,replies",
                    maxResults=100,
                    order="relevance",
                    videoId=videoID,
                    pageToken=npt
                )
            
            response = request.execute()

            if int(response['pageInfo']['totalResults']) == 0:
                break
            if "nextPageToken" in response:
                npt = response['nextPageToken']
            else:
                npt = None

            comments_total = len(response['items'])

            for j in range(comments_total):
                video_id.append(videoID)
                user_comment.append(
                    response['items'][j]['snippet']['topLevelComment']['snippet']['textOriginal'])
                user_name.append(
                    response['items'][j]['snippet']['topLevelComment']['snippet']['authorDisplayName'])
                user_likes_count.append(
                    response['items'][j]['snippet']['topLevelComment']['snippet']['likeCount'])

    except:
        #print('Comment disabled for ',search_results['videoId'][i])
        pass

### Make a csv for comments

In [4]:
# save into csv
pd.DataFrame({'Username': user_name, 'Comment': user_comment, 'Likes': user_likes_count,
             'Video ID': video_id}).to_csv('datasets/comments.csv', index=False)

In [5]:
# display the dataset
output = pd.read_csv("datasets/comments.csv")
display(output)

,Username,Comment,Likes,Video ID
0,Study IQ education,Click here to buy full UPSC course https://pla...,37,l1oobMY7CE4
1,Arun Kumar Sahoo,Think about their families... What they must b...,134,l1oobMY7CE4
2,Vishal Choudhury,Not a single day without the UP news.,550,l1oobMY7CE4
3,Nita Agarwal,Jaise-2 elections paas aayenge ye sab badhega ...,806,l1oobMY7CE4
4,Abodh S,"love your content, as it is totally unbiased :)",89,l1oobMY7CE4
...,...,...,...,...
265571,Rishi 10,"Farmer protest ""WINS"" central gov surrender",0,L0TsNO0h-qs
265572,Nilesh,"Modi finish Law, Healthcare, Economy, Educatio...",0,L0TsNO0h-qs
265573,Musarrat Khan,Shaheed kisano ko ( Salam),0,L0TsNO0h-qs
265574,narender kumar,अभी MSP पट्रोल और डिजल् बाकी है\n इस निकमी सर...,17,L0TsNO0h-qs


### Save comments in a text file

In [6]:
corpus = list(set(user_comment))
IO.save_text('datasets/comments.txt',corpus)